In [0]:
from pyspark.sql import functions as f, Window
from pyspark.sql.types import *
import pandas as pd
from datetime import date, datetime, timedelta
from IPython.core.display import HTML
import os, time
os.environ['TZ'] = 'America/Sao_Paulo'
time.tzset()
spark.conf.set("spark.sql.session.timeZone", "America/Sao_Paulo")

def capturaCotacao(moeda, dhMaxTbl):
  global getCotacao
  print(f'A data mínima da query é {dhMaxTbl.date().strftime("%Y-%m-%d")} (YYYY-MM-DD)')
  url = f'''https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/CotacaoMoedaPeriodo(moeda=@moeda,dataInicial=@dataInicial,dataFinalCotacao=@dataFinalCotacao)?@moeda=%27{moeda[1]}%27&@dataInicial=%27{dhMaxTbl.date().strftime("%m-%d-%Y")}%27&@dataFinalCotacao=%27{date.today().strftime("%m-%d-%Y")}%27&$skip=0&$format=text/csv&$select=paridadeCompra,paridadeVenda,cotacaoCompra,cotacaoVenda,dataHoraCotacao,tipoBoletim'''
  try:
    dfCotacao = spark.createDataFrame(pd.read_csv(url)).filter(f.col('tipoBoletim')=='Fechamento')\
                .withColumn('idMoeda', f.lit(moeda[0]))\
                .withColumn('simboloMoeda', f.lit(moeda[1]))\
                .withColumn('nomeMoeda', f.lit(moeda[2]))\
                .withColumn('tipoMoeda', f.lit(moeda[3]))\
                .select(f.col('simboloMoeda').cast('string'),\
                  f.col('idMoeda').cast('integer'),\
                  f.col('nomeMoeda').cast('string'),\
                  f.col('tipoMoeda').cast('string'),\
                  f.translate('paridadeCompra', ',', '.').cast('float').alias('paridadeCompra'),\
                  f.translate('paridadeVenda', ',', '.').cast('float').alias('paridadeVenda'),\
                  f.translate('cotacaoCompra', ',', '.').cast('float').alias('cotacaoCompra'),\
                  f.translate('cotacaoVenda', ',', '.').cast('float').alias('cotacaoVenda'),\
                  f.col('dataHoraCotacao').cast('timestamp').alias('dataHoraCotacao'),\
                  f.col('tipoBoletim').cast('string'))\
                .select('dataHoraCotacao', 'idMoeda', 'simboloMoeda', 'nomeMoeda', 'tipoMoeda', f.col('paridadeCompra').cast('float').alias('paridadeCompra'), f.col('paridadeVenda').cast('float').alias('paridadeVenda'), f.col('cotacaoCompra').cast('float').alias('cotacaoCompra'), f.col('cotacaoVenda').cast('float').alias('cotacaoVenda'), 'tipoBoletim')
    if getCotacao==None:
      getCotacao = dfCotacao
    else:
      getCotacao = getCotacao.union(dfCotacao)
  except Exception as e: 
    print(e)
    pass

In [0]:
moedas = [[6, 'AUD',	'Dólar australiano',	'B'],\
          [4, 'CAD',	'Dólar canadense',	'A'],\
          [11, 'CHF',	'Franco suíço',	'A'],\
          [9, 'DKK',	'Coroa dinamarquesa',	'A'],\
          [3, 'EUR',	'Euro',	'B'],\
          [8, 'GBP',	'Libra Esterlina',	'B'],\
          [33, 'JPY',	'Iene',	'A'],\
          [10, 'NOK',	'Coroa norueguesa',	'A'],\
          [7, 'SEK',	'Coroa sueca', 'A'],\
          [2, 'USD',	'Dólar dos Estados Unidos',	'A']]
getCotacao = None
try:
  dhMaxTblFechamento = spark.sql('SELECT max(dataHoraCotacao) as maxDataHoraCotacao FROM dwdCotacao where tipoBoletim="Fechamento"').collect()[0][0]
  dhMaxTblGeral = spark.sql('SELECT max(dataHoraCotacao) as maxDataHoraCotacao FROM dwdCotacao').collect()[0][0]
except:
  dhMaxTblFechamento = datetime(2000, 1, 1)
  dhMaxTblGeral = datetime(2000, 1, 1)
[capturaCotacao(i, dhMaxTblFechamento) for i in moedas]

A data mínima da query é 2021-05-05 (YYYY-MM-DD)
A data mínima da query é 2021-05-05 (YYYY-MM-DD)
A data mínima da query é 2021-05-05 (YYYY-MM-DD)
A data mínima da query é 2021-05-05 (YYYY-MM-DD)
A data mínima da query é 2021-05-05 (YYYY-MM-DD)
A data mínima da query é 2021-05-05 (YYYY-MM-DD)
A data mínima da query é 2021-05-05 (YYYY-MM-DD)
A data mínima da query é 2021-05-05 (YYYY-MM-DD)
A data mínima da query é 2021-05-05 (YYYY-MM-DD)
A data mínima da query é 2021-05-05 (YYYY-MM-DD)
Out[215]: [None, None, None, None, None, None, None, None, None, None]

In [0]:
cal = spark.createDataFrame(pd.DataFrame(pd.date_range(start=dhMaxTblFechamento.strftime("%m-%d-%Y"), end=date.today().strftime("%m-%d-%Y"), freq='D'), columns=['dhCotacao'])) \
.withColumn('dataCotacao', f.date_add(f.to_date('dhCotacao'), -1)) \
.withColumn('dhCotacao', f.date_add('dhCotacao', -1).cast('timestamp')) 
dfMoedas = spark.createDataFrame([i for i in moedas+[[1, 'BRL', 'Real Brasileiro', 'B']]], ['idMoeda', 'simboloMoeda', 'nomeMoeda', 'tipoMoeda'])
dfCalMoeda = cal.join(dfMoedas).sort(f.desc('dhCotacao'), 'simboloMoeda')
#display(dfCalMoeda)

dhCotacao,dataCotacao,idMoeda,simboloMoeda,nomeMoeda,tipoMoeda
2021-05-08T00:00:00.000-0300,2021-05-08,6,AUD,Dólar australiano,B
2021-05-08T00:00:00.000-0300,2021-05-08,1,BRL,Real Brasileiro,B
2021-05-08T00:00:00.000-0300,2021-05-08,4,CAD,Dólar canadense,A
2021-05-08T00:00:00.000-0300,2021-05-08,11,CHF,Franco suíço,A
2021-05-08T00:00:00.000-0300,2021-05-08,9,DKK,Coroa dinamarquesa,A
2021-05-08T00:00:00.000-0300,2021-05-08,3,EUR,Euro,B
2021-05-08T00:00:00.000-0300,2021-05-08,8,GBP,Libra Esterlina,B
2021-05-08T00:00:00.000-0300,2021-05-08,33,JPY,Iene,A
2021-05-08T00:00:00.000-0300,2021-05-08,10,NOK,Coroa norueguesa,A
2021-05-08T00:00:00.000-0300,2021-05-08,7,SEK,Coroa sueca,A


In [0]:
windowSemana = Window.partitionBy('simboloMoeda').orderBy('dataHoraCotacao').rowsBetween(-7, -1)
getCotacaoT1 = \
getCotacao\
.union(getCotacao.filter(f.col('simboloMoeda')=='USD')\
        .withColumn('idMoeda', f.lit(1))\
        .withColumn('simboloMoeda', f.lit('BRL'))\
        .withColumn('nomeMoeda', f.lit('Real Brasileiro'))\
        .withColumn('tipoMoeda', f.lit('B'))\
        .withColumn('estrutura', f.struct('paridadeCompra', 'paridadeVenda', 'cotacaoCompra', 'cotacaoVenda'))\
        .withColumn('paridadeCompra', 1/f.col('estrutura').cotacaoCompra)\
        .withColumn('paridadeVenda', 1/f.col('estrutura').cotacaoVenda)\
        .withColumn('cotacaoCompra', (f.col('estrutura').paridadeCompra))\
        .withColumn('cotacaoVenda', (f.col('estrutura').paridadeVenda))\
        .drop('estrutura'))\
.withColumn('dataCotacao', f.to_date('dataHoraCotacao'))\
.join(f.broadcast(dfCalMoeda), ['dataCotacao', 'idMoeda', 'simboloMoeda', 'nomeMoeda', 'tipoMoeda'], 'outer')\
.withColumn('dataHoraCotacao', f.when(f.col('dataHoraCotacao').isNull(), f.col('dhCotacao')).otherwise(f.col('dataHoraCotacao'))) \
.drop('dhCotacao', 'dataCotacao') \
.withColumn('paridadeCompra', f.when(f.col('paridadeCompra').isNull(), f.last('paridadeCompra', ignorenulls=True).over(windowSemana)).otherwise(f.col('paridadeCompra')))\
.withColumn('paridadeVenda', f.when(f.col('paridadeVenda').isNull(), f.last('paridadeVenda', ignorenulls=True).over(windowSemana)).otherwise(f.col('paridadeVenda')))\
.withColumn('cotacaoCompra', f.when(f.col('cotacaoCompra').isNull(), f.last('cotacaoCompra', ignorenulls=True).over(windowSemana)).otherwise(f.col('cotacaoCompra')))\
.withColumn('cotacaoVenda', f.when(f.col('cotacaoVenda').isNull(), f.last('cotacaoVenda', ignorenulls=True).over(windowSemana)).otherwise(f.col('cotacaoVenda')))\
.withColumn('tipoBoletim', f.when(f.col('tipoBoletim').isNull(), f.lit('Preenchimento')).otherwise(f.col('tipoBoletim')))\
.sort(f.desc('dataHoraCotacao'), 'simboloMoeda')\
.withColumn('CotacaoCompraUSD', f.when(f.col('tipoMoeda')=='A', 1/f.col('paridadeCompra')).otherwise(f.col('paridadeCompra')))\
.withColumn('CotacaoVendaUSD', f.when(f.col('tipoMoeda')=='A', 1/f.col('paridadeVenda')).otherwise(f.col('paridadeVenda')))\
.select('dataHoraCotacao', 'idMoeda', 'simboloMoeda', 'nomeMoeda', 'tipoMoeda', f.col('cotacaoCompraUSD').cast('float').alias('cotacaoCompraUSD'), f.col('cotacaoVendaUSD').cast('float').alias('cotacaoVendaUSD'),\
        f.col('cotacaoCompra').cast('float').alias('cotacaoCompraBRL'), f.col('cotacaoVenda').cast('float').alias('cotacaoVendaBRL'), 'tipoBoletim')\
.filter(f.col('dataHoraCotacao')>dhMaxTblGeral)\
.filter(f.col('cotacaoVendaBRL').isNotNull())
display(getCotacaoT1)

dataHoraCotacao,idMoeda,simboloMoeda,nomeMoeda,tipoMoeda,cotacaoCompraUSD,cotacaoVendaUSD,cotacaoCompraBRL,cotacaoVendaBRL,tipoBoletim
2021-05-08T00:00:00.000-0300,6,AUD,Dólar australiano,B,0.7859,0.786,4.1033,4.1043,Preenchimento
2021-05-08T00:00:00.000-0300,1,BRL,Real Brasileiro,B,0.19153053,0.1915085,1.0,1.0,Preenchimento
2021-05-08T00:00:00.000-0300,4,CAD,Dólar canadense,A,0.8241985,0.8239268,4.3018,4.3037,Preenchimento
2021-05-08T00:00:00.000-0300,11,CHF,Franco suíço,A,1.1101243,1.1097548,5.7941,5.7967,Preenchimento
2021-05-08T00:00:00.000-0300,9,DKK,Coroa dinamarquesa,A,0.16361256,0.16355905,0.854,0.8543,Preenchimento
2021-05-08T00:00:00.000-0300,3,EUR,Euro,B,1.2164,1.2165,6.3509,6.3522,Preenchimento
2021-05-08T00:00:00.000-0300,8,GBP,Libra Esterlina,B,1.3998,1.4002,7.3085,7.3114,Preenchimento
2021-05-08T00:00:00.000-0300,33,JPY,Iene,A,0.0092157405,0.009213193,0.0481,0.04812,Preenchimento
2021-05-08T00:00:00.000-0300,10,NOK,Coroa norueguesa,A,0.12182494,0.121765606,0.6358,0.6361,Preenchimento
2021-05-08T00:00:00.000-0300,7,SEK,Coroa sueca,A,0.12029207,0.120248675,0.6278,0.6281,Preenchimento


In [0]:
## Grava a tabela Hive
getCotacaoT1.sort(f.desc('dataHoraCotacao'), 'simboloMoeda').write.partitionBy("simboloMoeda").saveAsTable('default.dwdCotacao', mode='append')

In [0]:
display(spark.sql('select * from default.dwdCotacao order by dataHoraCotacao desc, simboloMoeda'))

dataHoraCotacao,idMoeda,simboloMoeda,nomeMoeda,tipoMoeda,cotacaoCompraUSD,cotacaoVendaUSD,cotacaoCompraBRL,cotacaoVendaBRL,tipoBoletim
2021-05-08T00:00:00.000-0300,6,AUD,Dólar australiano,B,0.7859,0.786,4.1033,4.1043,Preenchimento
2021-05-08T00:00:00.000-0300,1,BRL,Real Brasileiro,B,0.19153053,0.1915085,1.0,1.0,Preenchimento
2021-05-08T00:00:00.000-0300,4,CAD,Dólar canadense,A,0.8241985,0.8239268,4.3018,4.3037,Preenchimento
2021-05-08T00:00:00.000-0300,11,CHF,Franco suíço,A,1.1101243,1.1097548,5.7941,5.7967,Preenchimento
2021-05-08T00:00:00.000-0300,9,DKK,Coroa dinamarquesa,A,0.16361256,0.16355905,0.854,0.8543,Preenchimento
2021-05-08T00:00:00.000-0300,3,EUR,Euro,B,1.2164,1.2165,6.3509,6.3522,Preenchimento
2021-05-08T00:00:00.000-0300,8,GBP,Libra Esterlina,B,1.3998,1.4002,7.3085,7.3114,Preenchimento
2021-05-08T00:00:00.000-0300,33,JPY,Iene,A,0.0092157405,0.009213193,0.0481,0.04812,Preenchimento
2021-05-08T00:00:00.000-0300,10,NOK,Coroa norueguesa,A,0.12182494,0.121765606,0.6358,0.6361,Preenchimento
2021-05-08T00:00:00.000-0300,7,SEK,Coroa sueca,A,0.12029207,0.120248675,0.6278,0.6281,Preenchimento


In [0]:
#spark.sql('delete from dwdCotacao where dataHoraCotacao >= "2021-05-06"').show()

In [0]:
%%markdown
drop table dwdCotacao

<IPython.core.display.Markdown object>